# FeedForward Products TensorFlow

In [1]:
from scipy import sparse

import numpy as np
import tensorflow as tf

In [2]:
user_prod_matrix = sparse.load_npz("data/user_prod_matrix.npz")
target_matrix = sparse.load_npz("data/target_prod_matrix.npz")

In [3]:
X_train = user_prod_matrix.todense()
Y_train = target_matrix.todense()

In [4]:
X_train.shape

(206210, 36055)

In [5]:
Y_train.shape

(206210, 36055)

In [6]:
np.asarray(X_train[0]).shape

(1, 36055)

# Define Network

In [27]:
def train_model(hidden_nodes, num_iters):
    
    tf.reset_default_graph()
    
    # input and output
    X = tf.sparse_placeholder(shape=X_train.shape, dtype=tf.int32, name='X')
    Y = tf.sparse_placeholder(shape=Y_train.shape, dtype=tf.int32, name='Y')
    
    # weights
    W1 = tf.Variable(np.random.rand(X_train.shape[1], hidden_nodes), dtype=tf.float64)
    W2 = tf.Variable(np.random.rand(hidden_nodes, 100), dtype=tf.float64)
    
    # neural network
    A1 = tf.nn.relu(tf.matmul(X, W1))
    y_est = tf.nn.relu(tf.matmul(A1, W2))
    
    # loss
    deltas = tf.square(y_est - Y)
    loss = tf.reduce_sum(deltas)
    
    # optimize
    optimizer = tf.train.GradientDescentOptimizer(0.005)
    train = optimizer.minimize(loss)

    # initialize session
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    for i in range(num_iters):
        feed_dict = {X: tf.SparseTensorValue(X_train), y: tf.SparseTensorValue(Y_train)}
        sess.run(train, feed_dict=feed_dict)
        loss_plot[hidden_nodes].append(sess.run(loss, feed_dict=feed_dict))

        weights1 = sess.run(W1)
        weights2 = sess.run(W2)

    print("loss (hidden nodes: %d, iterations: %d): %.2f" % (hidden_nodes, num_iters, loss_plot[hidden_nodes][-1]))
    sess.close()
    
    return loss_plot[hidden_nodes][-1]
 

In [28]:
train_model(206210, 5)

MemoryError: 

##MemoryError

# Generator Trainer

In [21]:
def get_batch():
    for x, y in zip(X_train, Y_train):
        yield (np.array(x), np.array(y))

In [24]:
x, y = next(iter(get_batch()))

In [26]:
x.shape

(1, 36055)

# Network with Generator

In [29]:
def train_model_generator(hidden_nodes, num_iters):
    
    tf.reset_default_graph()
    
    # input and output
    X = tf.placeholder(shape=(1, X_train.shape[1]), dtype=tf.int32, name='X')
    Y = tf.placeholder(shape=(1, Y_train.shape[1]), dtype=tf.int32, name='Y')
    
    # weights
    W1 = tf.Variable(np.random.rand(X_train.shape[1], hidden_nodes), dtype=tf.float64)
    W2 = tf.Variable(np.random.rand(hidden_nodes, 100), dtype=tf.float64)
    
    # neural network
    A1 = tf.nn.relu(tf.matmul(X, W1))
    y_est = tf.nn.relu(tf.matmul(A1, W2))
    
    # loss
    deltas = tf.square(y_est - Y)
    loss = tf.reduce_sum(deltas)
    
    # optimize
    optimizer = tf.train.GradientDescentOptimizer(0.005)
    train = optimizer.minimize(loss)

    # initialize session
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    for i in range(num_iters):
        for X_batch, y_batch in get_batch():
            feed_dict = {X: X_batch, y: y_batch}
            sess.run(train, feed_dict=feed_dict)
            loss_plot[hidden_nodes].append(sess.run(loss, feed_dict=feed_dict))
            weights1 = sess.run(W1)
            weights2 = sess.run(W2)

    print("loss (hidden nodes: %d, iterations: %d): %.2f" % (hidden_nodes, num_iters, loss_plot[hidden_nodes][-1]))
    sess.close()
    
    return loss_plot[hidden_nodes][-1]
 

In [31]:
train_model_generator(X_train.shape[1], 5)

MemoryError: 